In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from zipfile import ZipFile
import os
import pandas as pd
import csv
import cv2

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

import numpy as np
from PIL import Image
import tensorflow as tf
from torch.utils.data import DataLoader, TensorDataset
import torch

# Dataset & make CSV

In [ ]:
# 구글 드라이브에 업로드된 압축 파일 경로
zip_file_path = '/content/drive/MyDrive/Colab Notebooks/DL/Homework/archive.zip'

# 압축을 해제할 디렉토리 경로
extracted_folder_path = '/content/drive/MyDrive/Colab Notebooks/DL/Homework'

# 압축 파일 해제
with ZipFile(zip_file_path, 'r') as zip_ref:
  zip_ref.extractall(extracted_folder_path)

# 압축 해제 후의 파일 목록 출력
extracted_files = os.listdir(extracted_folder_path)
print("파일 목록:", extracted_files)

파일 목록: ['modeling.ipynb', 'archive.zip', 'Hair Diseases - Final']


In [ ]:
directory_path = '/content/drive/MyDrive/Colab Notebooks/DL/Homework/Hair Diseases - Final/test'
file_list = os.listdir(directory_path)

print("디렉토리 내의 파일 목록:")
for file_name in file_list:
    print(file_name)

diseases_list = [file_name for file_name in file_list]
print(diseases_list)

디렉토리 내의 파일 목록:
Alopecia Areata
Contact Dermatitis
Folliculitis
Head Lice
Lichen Planus
Male Pattern Baldness
Psoriasis
Seborrheic Dermatitis
Telogen Effluvium
Tinea Capitis
['Alopecia Areata', 'Contact Dermatitis', 'Folliculitis', 'Head Lice', 'Lichen Planus', 'Male Pattern Baldness', 'Psoriasis', 'Seborrheic Dermatitis', 'Telogen Effluvium', 'Tinea Capitis']


In [ ]:
# 특정 디렉토리 경로
for d in diseases_list:
  train_path = '/content/drive/MyDrive/Colab Notebooks/DL/Homework/Hair Diseases - Final/train/'+ d
  test_path =  '/content/drive/MyDrive/Colab Notebooks/DL/Homework/Hair Diseases - Final/test/'+ d
  val_path = '/content/drive/MyDrive/Colab Notebooks/DL/Homework/Hair Diseases - Final/val/'+ d

  # 디렉토리 내의 파일 목록을 가져옴
  file_list1 = os.listdir(train_path)
  file_list2 = os.listdir(test_path)
  file_list3 = os.listdir(val_path)

  # 파일 개수 출력
  file_count1 = len(file_list1)
  file_count2 = len(file_list2)
  file_count3 = len(file_list3)
  print(f"train_'{d}' 디렉토리에 있는 파일 개수: {file_count1}")
  print(f"test_'{d}' 디렉토리에 있는 파일 개수: {file_count2}")
  print(f"val_'{d}' 디렉토리에 있는 파일 개수: {file_count3}")

train_'Alopecia Areata' 디렉토리에 있는 파일 개수: 960
test_'Alopecia Areata' 디렉토리에 있는 파일 개수: 120
val_'Alopecia Areata' 디렉토리에 있는 파일 개수: 120
train_'Contact Dermatitis' 디렉토리에 있는 파일 개수: 960
test_'Contact Dermatitis' 디렉토리에 있는 파일 개수: 120
val_'Contact Dermatitis' 디렉토리에 있는 파일 개수: 120
train_'Folliculitis' 디렉토리에 있는 파일 개수: 960
test_'Folliculitis' 디렉토리에 있는 파일 개수: 120
val_'Folliculitis' 디렉토리에 있는 파일 개수: 120
train_'Head Lice' 디렉토리에 있는 파일 개수: 960
test_'Head Lice' 디렉토리에 있는 파일 개수: 120
val_'Head Lice' 디렉토리에 있는 파일 개수: 120
train_'Lichen Planus' 디렉토리에 있는 파일 개수: 960
test_'Lichen Planus' 디렉토리에 있는 파일 개수: 120
val_'Lichen Planus' 디렉토리에 있는 파일 개수: 120
train_'Male Pattern Baldness' 디렉토리에 있는 파일 개수: 960
test_'Male Pattern Baldness' 디렉토리에 있는 파일 개수: 120
val_'Male Pattern Baldness' 디렉토리에 있는 파일 개수: 120
train_'Psoriasis' 디렉토리에 있는 파일 개수: 960
test_'Psoriasis' 디렉토리에 있는 파일 개수: 120
val_'Psoriasis' 디렉토리에 있는 파일 개수: 120
train_'Seborrheic Dermatitis' 디렉토리에 있는 파일 개수: 960
test_'Seborrheic Dermatitis' 디렉토리에 있는 파일 개수: 120
val_'Seborrheic Dermat

- train: 960개
- test: 120개
- validation: 120개

총 960 + 120 + 120 = 12000개 dataset

### train dataset

In [ ]:
# 디렉토리 경로
train_path = '/content/drive/MyDrive/Colab Notebooks/DL/Homework/Hair Diseases - Final/train'

# CSV 파일 경로
csv_file_path = '/content/drive/MyDrive/Colab Notebooks/DL/Homework/Hair Diseases - Final/train_images.csv'

# 질병 리스트
disease_list = ['Alopecia Areata', 'Contact Dermatitis', 'Folliculitis', 'Head Lice',
                'Lichen Planus', 'Male Pattern Baldness', 'Psoriasis',
                'Seborrheic Dermatitis', 'Telogen Effluvium', 'Tinea Capitis']

# CSV 파일을 작성할 때 사용할 헤더
csv_header = ['File Name'] + disease_list

# CSV 파일 열기
with open(csv_file_path, 'w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)
    # 헤더 작성
    csv_writer.writerow(csv_header)
    # 디렉토리 순회
    for disease_folder in os.listdir(train_path):
        disease_path = os.path.join(train_path, disease_folder)
        # 디렉토리인지 확인하고 jpg 파일 수집
        if os.path.isdir(disease_path):
            for file_name in os.listdir(disease_path):
                if file_name.endswith('.jpg'):
                    # 파일명에서 .jpg 제거 후 기록
                    row = [file_name[:-4]] + [1 if disease_folder == disease else 0 for disease in disease_list]
                    csv_writer.writerow(row)

print(f"CSV 파일이 생성되었습니다. 경로: {csv_file_path}")

# 생성된 CSV 파일의 경로
csv_train_path = '/content/drive/MyDrive/Colab Notebooks/DL/Homework/Hair Diseases - Final/train_images.csv'

# CSV 파일 읽기
df_train = pd.read_csv(csv_train_path)

# 데이터프레임 출력
df_train

CSV 파일이 생성되었습니다. 경로: /content/drive/MyDrive/Colab Notebooks/DL/Homework/Hair Diseases - Final/train_images.csv


,File Name,Alopecia Areata,Contact Dermatitis,Folliculitis,Head Lice,Lichen Planus,Male Pattern Baldness,Psoriasis,Seborrheic Dermatitis,Telogen Effluvium,Tinea Capitis
0,alopecia_areata_0002,1,0,0,0,0,0,0,0,0,0
1,alopecia_areata_0003,1,0,0,0,0,0,0,0,0,0
2,alopecia_areata_0004,1,0,0,0,0,0,0,0,0,0
3,alopecia_areata_0005,1,0,0,0,0,0,0,0,0,0
4,alopecia_areata_0007,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
9595,tinea_capitis_1196,0,0,0,0,0,0,0,0,0,1
9596,tinea_capitis_1197,0,0,0,0,0,0,0,0,0,1
9597,tinea_capitis_1198,0,0,0,0,0,0,0,0,0,1
9598,tinea_capitis_1199,0,0,0,0,0,0,0,0,0,1


### test dataset

In [ ]:
import os
import csv
import pandas as pd

# 디렉토리 경로
test_path = '/content/drive/MyDrive/Colab Notebooks/DL/Homework/Hair Diseases - Final/test'

# CSV 파일 경로
csv_file_path = '/content/drive/MyDrive/Colab Notebooks/DL/Homework/Hair Diseases - Final/test_images.csv'

# 질병 리스트
disease_list = ['Alopecia Areata', 'Contact Dermatitis', 'Folliculitis', 'Head Lice',
                'Lichen Planus', 'Male Pattern Baldness', 'Psoriasis',
                'Seborrheic Dermatitis', 'Telogen Effluvium', 'Tinea Capitis']

# CSV 파일을 작성할 때 사용할 헤더
csv_header = ['File Name'] + disease_list

# CSV 파일 열기
with open(csv_file_path, 'w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)
    # 헤더 작성
    csv_writer.writerow(csv_header)
    # 디렉토리 순회
    for disease_folder in os.listdir(test_path):
        disease_path = os.path.join(test_path, disease_folder)
        # 디렉토리인지 확인하고 jpg 파일 수집
        if os.path.isdir(disease_path):
            for file_name in os.listdir(disease_path):
                if file_name.endswith('.jpg'):
                    # 파일명에서 .jpg 제거 후 기록
                    row = [file_name[:-4]] + [1 if disease_folder == disease else 0 for disease in disease_list]
                    csv_writer.writerow(row)

print(f"CSV 파일이 생성되었습니다. 경로: {csv_file_path}")

# 생성된 CSV 파일의 경로
csv_test_path = '/content/drive/MyDrive/Colab Notebooks/DL/Homework/Hair Diseases - Final/test_images.csv'

# CSV 파일 읽기
df_test = pd.read_csv(csv_test_path)

# 데이터프레임 출력
df_test

CSV 파일이 생성되었습니다. 경로: /content/drive/MyDrive/Colab Notebooks/DL/Homework/Hair Diseases - Final/test_images.csv


,File Name,Alopecia Areata,Contact Dermatitis,Folliculitis,Head Lice,Lichen Planus,Male Pattern Baldness,Psoriasis,Seborrheic Dermatitis,Telogen Effluvium,Tinea Capitis
0,alopecia_areata_0026,1,0,0,0,0,0,0,0,0,0
1,alopecia_areata_0036,1,0,0,0,0,0,0,0,0,0
2,alopecia_areata_0037,1,0,0,0,0,0,0,0,0,0
3,alopecia_areata_0051,1,0,0,0,0,0,0,0,0,0
4,alopecia_areata_0062,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
1195,tinea_capitis_1147,0,0,0,0,0,0,0,0,0,1
1196,tinea_capitis_1154,0,0,0,0,0,0,0,0,0,1
1197,tinea_capitis_1162,0,0,0,0,0,0,0,0,0,1
1198,tinea_capitis_1170,0,0,0,0,0,0,0,0,0,1


###  validation dataset

In [ ]:
import os
import csv
import pandas as pd

# 디렉토리 경로
val_path = '/content/drive/MyDrive/Colab Notebooks/DL/Homework/Hair Diseases - Final/val'

# CSV 파일 경로
csv_file_path = '/content/drive/MyDrive/Colab Notebooks/DL/Homework/Hair Diseases - Final/val_images.csv'

# 질병 리스트
disease_list = ['Alopecia Areata', 'Contact Dermatitis', 'Folliculitis', 'Head Lice',
                'Lichen Planus', 'Male Pattern Baldness', 'Psoriasis',
                'Seborrheic Dermatitis', 'Telogen Effluvium', 'Tinea Capitis']

# CSV 파일을 작성할 때 사용할 헤더
csv_header = ['File Name'] + disease_list

# CSV 파일 열기
with open(csv_file_path, 'w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)
    # 헤더 작성
    csv_writer.writerow(csv_header)
    # 디렉토리 순회
    for disease_folder in os.listdir(val_path):
        disease_path = os.path.join(val_path, disease_folder)
        # 디렉토리인지 확인하고 jpg 파일 수집
        if os.path.isdir(disease_path):
            for file_name in os.listdir(disease_path):
                if file_name.endswith('.jpg'):
                    # 파일명에서 .jpg 제거 후 기록
                    row = [file_name[:-4]] + [1 if disease_folder == disease else 0 for disease in disease_list]
                    csv_writer.writerow(row)

print(f"CSV 파일이 생성되었습니다. 경로: {csv_file_path}")

# 생성된 CSV 파일의 경로
csv_val_path = '/content/drive/MyDrive/Colab Notebooks/DL/Homework/Hair Diseases - Final/val_images.csv'

# CSV 파일 읽기
df_val = pd.read_csv(csv_val_path)

# 데이터프레임 출력
df_val

CSV 파일이 생성되었습니다. 경로: /content/drive/MyDrive/Colab Notebooks/DL/Homework/Hair Diseases - Final/val_images.csv


,File Name,Alopecia Areata,Contact Dermatitis,Folliculitis,Head Lice,Lichen Planus,Male Pattern Baldness,Psoriasis,Seborrheic Dermatitis,Telogen Effluvium,Tinea Capitis
0,alopecia_areata_0001,1,0,0,0,0,0,0,0,0,0
1,alopecia_areata_0006,1,0,0,0,0,0,0,0,0,0
2,alopecia_areata_0016,1,0,0,0,0,0,0,0,0,0
3,alopecia_areata_0029,1,0,0,0,0,0,0,0,0,0
4,alopecia_areata_0033,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
1195,tinea_capitis_1151,0,0,0,0,0,0,0,0,0,1
1196,tinea_capitis_1173,0,0,0,0,0,0,0,0,0,1
1197,tinea_capitis_1177,0,0,0,0,0,0,0,0,0,1
1198,tinea_capitis_1185,0,0,0,0,0,0,0,0,0,1


In [ ]:
# 각 열에서 값이 1인 개수 출력
one_counts = df_val.iloc[:, 1:].sum()  # 첫 번째 열 제외
print("각 열에서 1의 개수:")
print(one_counts)

각 열에서 1의 개수:
Alopecia Areata          120
Contact Dermatitis       120
Folliculitis             120
Head Lice                120
Lichen Planus            120
Male Pattern Baldness    120
Psoriasis                120
Seborrheic Dermatitis    120
Telogen Effluvium        120
Tinea Capitis            120
dtype: int64


In [ ]:
# 각 열에서 값이 1인 개수 출력
one_counts = df_train.iloc[:, 1:].sum()  # 첫 번째 열 제외
print("각 열에서 1의 개수:")
print(one_counts)

각 열에서 1의 개수:
Alopecia Areata          960
Contact Dermatitis       960
Folliculitis             960
Head Lice                960
Lichen Planus            960
Male Pattern Baldness    960
Psoriasis                960
Seborrheic Dermatitis    960
Telogen Effluvium        960
Tinea Capitis            960
dtype: int64


In [ ]:
# 각 열에서 값이 1인 개수 출력
one_counts = df_test.iloc[:, 1:].sum()  # 첫 번째 열 제외
print("각 열에서 1의 개수:")
print(one_counts)

각 열에서 1의 개수:
Alopecia Areata          120
Contact Dermatitis       120
Folliculitis             120
Head Lice                120
Lichen Planus            120
Male Pattern Baldness    120
Psoriasis                120
Seborrheic Dermatitis    120
Telogen Effluvium        120
Tinea Capitis            120
dtype: int64


# Data Preprocessing

### train

In [ ]:
# IMG_SIZE 설정
IMG_SIZE = 256
train_path = '/content/drive/MyDrive/Colab Notebooks/DL/Homework/Hair Diseases - Final/train'

data = []
labels = []

# 질병 리스트
disease_list = ['Alopecia Areata', 'Contact Dermatitis', 'Folliculitis', 'Head Lice',
                'Lichen Planus', 'Male Pattern Baldness', 'Psoriasis',
                'Seborrheic Dermatitis', 'Telogen Effluvium', 'Tinea Capitis']

for disease_folder in os.listdir(train_path):
    disease_path = os.path.join(train_path, disease_folder)

    # 디렉토리인지 확인하고 jpg 파일 수집
    if os.path.isdir(disease_path):
        for file_name in os.listdir(disease_path):
            if file_name.endswith('.jpg'):
                # 이미지 읽기
                img_path = os.path.join(disease_path, file_name)
                img = Image.open(img_path)
                # 이미지 리사이즈
                img = img.resize((IMG_SIZE, IMG_SIZE))
                # PIL 이미지를 numpy 배열로 변환
                img_np = np.array(img)
                # 노이즈 제거
                median_filtered_image = cv2.medianBlur(img_np, 3)
                one_img = np.asarray(np.float32(median_filtered_image))
                norm_img = one_img / 255.0

                # 데이터에 추가
                data.append(norm_img)

                # 레이블 추가
                label = [1 if disease_folder == disease else 0 for disease in disease_list]
                labels.append(label)  # 1차원 리스트로 변경

# 데이터와 레이블을 numpy 배열로 변환
data = np.array(data)
labels = np.array(labels)

# 데이터 확인
print("Data shape:", data.shape)
print("Labels shape:", labels.shape)

Data shape: (9600, 256, 256, 3)
Labels shape: (9600, 10)


In [ ]:
data = data.transpose((0, 3, 1, 2))

In [ ]:
# 데이터 확인
print("Data shape:", data.shape)
print("Labels shape:", labels.shape)

Data shape: (9600, 3, 256, 256)
Labels shape: (9600, 10)


### val

In [ ]:
val_path = '/content/drive/MyDrive/Colab Notebooks/DL/Homework/Hair Diseases - Final/val'

val_data = []
val_labels = []

for disease_folder in os.listdir(val_path):
    disease_path = os.path.join(val_path, disease_folder)

    # 디렉토리인지 확인하고 jpg 파일 수집
    if os.path.isdir(disease_path):
        for file_name in os.listdir(disease_path):
            if file_name.endswith('.jpg'):
                # 이미지 읽기
                img_path = os.path.join(disease_path, file_name)
                img = Image.open(img_path)
                # 이미지 리사이즈
                img = img.resize((IMG_SIZE, IMG_SIZE))
                # PIL 이미지를 numpy 배열로 변환
                img_np = np.array(img)
                # 노이즈 제거
                median_filtered_image = cv2.medianBlur(img_np, 3)
                one_img = np.asarray(np.float32(median_filtered_image))
                norm_img = one_img / 255.0

                # 데이터에 추가
                val_data.append(norm_img)

                # 레이블 추가
                label = [1 if disease_folder == disease else 0 for disease in disease_list]
                val_labels.append(label)  # 1차원 리스트로 변경


# 데이터와 레이블을 numpy 배열로 변환
val_data = np.array(val_data)
val_labels = np.array(val_labels)

# 데이터 확인
print("Validation Data shape:", val_data.shape)
print("Validation Labels shape:", val_labels.shape)

Validation Data shape: (1200, 256, 256, 3)
Validation Labels shape: (1200, 10)


In [ ]:
val_data = val_data.transpose((0, 3, 1, 2))

In [ ]:
# 데이터 확인
print("Validation Data shape:", val_data.shape)
print("Validation Labels shape:", val_labels.shape)

Validation Data shape: (1200, 3, 256, 256)
Validation Labels shape: (1200, 10)


In [ ]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, data, labels):
        self.data = np.array(data)
        self.labels = [label.tolist() for label in labels]

    def __len__(self):
        return self.data.shape[0]

    def __getitem__(self, idx):
        images = torch.FloatTensor(self.data[idx])
        labels = torch.FloatTensor(self.labels[idx])

        return images, labels.squeeze(0)  # 2차원에서 1차원으로 변경

In [ ]:
# 데이터셋 인스턴스 생성
train_dataset = CustomDataset(data, labels)
val_dataset = CustomDataset(val_data, val_labels)

# DataLoader 생성
batch_size_train = 64
batch_size_val = 8

train_loader = DataLoader(train_dataset, batch_size=batch_size_train, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size_val, shuffle=False)

# Model Architecture & Train

In [ ]:
!pip install torchsummaryX

In [ ]:
!pip install torchsummary --upgrade

In [ ]:
!pip install pretrainedmodels

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-py3-none-any.whl size=60944 sha256=d271e663ae3203280d702e743d695597eec82c7684cebb9ce997a32c3ce11419
  Stored in directory: /root/.cache/pip/wheels/35/cb/a5/8f534c60142835bfc889f9a482e4a67e0b817032d9c6883b64
Successfully built pretrainedmodels


In [ ]:
!pip install timm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.6/51.6 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 29.1 MB/s eta 0:00:00


In [ ]:
# PyTorch
from torch import nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
import torch.utils.data as data
from torch.utils.data import DataLoader, TensorDataset
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
from torch.optim import lr_scheduler
from torchvision import utils
from torchsummary import summary
from torchsummaryX import summary
from torchvision.models import resnet50
from torchvision.models import resnet101

In [ ]:
from torchvision.models import resnet50, ResNet50_Weights

# 모델 정의
class MultiBinaryClassificationModel(nn.Module):
    def __init__(self, num_classes=10) -> None:
        super(MultiBinaryClassificationModel, self).__init__()
        self.resnet = resnet50(weights=ResNet50_Weights.DEFAULT)  # weights 인수 사용
        self.classifier = nn.Linear(1000, num_classes)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.resnet(x)
        x = self.classifier(x)
        x = self.sigmoid(x)
        return x

In [ ]:
import torch
from torchsummary import summary

# 모델 인스턴스 생성
num_classes = 10  # 클래스 개수에 맞게 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = MultiBinaryClassificationModel(num_classes=num_classes).to(device)

# 입력 데이터 생성
input_data = torch.randn((1, 3, 256, 256)).to(device)  # (배치 크기, 채널 수, 높이, 너비)

# 모델 요약 출력
summary(model, input_data.shape[1:])  # input_data의 크기에서 배치 크기 제외

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 205MB/s]


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 128, 128]           9,408
       BatchNorm2d-2         [-1, 64, 128, 128]             128
              ReLU-3         [-1, 64, 128, 128]               0
         MaxPool2d-4           [-1, 64, 64, 64]               0
            Conv2d-5           [-1, 64, 64, 64]           4,096
       BatchNorm2d-6           [-1, 64, 64, 64]             128
              ReLU-7           [-1, 64, 64, 64]               0
            Conv2d-8           [-1, 64, 64, 64]          36,864
       BatchNorm2d-9           [-1, 64, 64, 64]             128
             ReLU-10           [-1, 64, 64, 64]               0
           Conv2d-11          [-1, 256, 64, 64]          16,384
      BatchNorm2d-12          [-1, 256, 64, 64]             512
           Conv2d-13          [-1, 256, 64, 64]          16,384
      BatchNorm2d-14          [-1, 256,

In [ ]:
torch.cuda.empty_cache()

In [ ]:
# 모델 학습 함수 정의
def run_train(model):
    # 모델을 학습 모드로 전환
    model.train()

    # 손실 함수와 최적화 기법 정의
    criterion = nn.BCELoss()  # 이진 분류를 위한 손실 함수
    optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adam 최적화 기법 사용

    total_loss = 0  # 총 손실 초기화
    correct = 0     # 맞춘 예측 수 초기화
    incorrect = 0   # 틀린 예측 수 초기화

    # 학습 데이터 로더를 통해 입력과 레이블 반복
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        # 기울기 초기화
        optimizer.zero_grad()
        # 모델을 통해 출력값 계산
        outputs = model(inputs)
        # 손실 계산
        loss = criterion(outputs, labels)
        # 기울기 계산
        loss.backward()
        # 파라미터 업데이트
        optimizer.step()
        # 총 손실에 현재 손실 추가
        total_loss += loss.item()
        # 예측 결과를 0.5 기준으로 이진화
        predictions = outputs > 0.5
        # 맞춘 예측 수와 틀린 예측 수 계산
        correct += (predictions == labels).sum().item()
        incorrect += (predictions != labels).sum().item()

    # 정확도 계산
    accuracy = correct / (correct + incorrect)

    # 평균 손실 계산
    average_loss = total_loss / len(train_loader)

    # 정확도, 평균 손실, 예측 결과, 실제 레이블 반환
    return accuracy, average_loss, predictions, labels

In [ ]:
# Validation
def run_valid(model):
    model.eval()
    criterion = nn.MSELoss()

    total_loss = 0
    correct = 0
    incorrect = 0

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            total_loss += loss.item()

            predictions = outputs > 0.5
            correct += (predictions == labels).sum().item()
            incorrect += (predictions != labels).sum().item()

    accuracy = correct / (correct + incorrect)
    average_loss = total_loss / len(val_loader)

    return accuracy, average_loss, predictions, labels

In [ ]:
best_accuracy = 0
best_epoch = 0

# Learning
acc_t = []
acc_v = []
prediction_v = []
label_v = []

epoch = 100  # Number of epochs
patience = 10  # Number of epochs to wait for improvement
counter = 0  # Counter for early stopping

print('***START EPOCH***')

for E in range(epoch):
    print('# Epoch: {} / {}'.format(E+1, epoch))
    accuracy_t, loss_t, pred_t, truth_t = run_train(model)
    accuracy_v, loss_v, pred_v, truth_v = run_valid(model)
    prediction_v.append(pred_v)
    label_v.append(truth_v)

    print('Train Accuracy: {}'.format(accuracy_t))
    print('Validation Accuracy: {}'.format(accuracy_v))
    acc_t.append(accuracy_t)
    acc_v.append(accuracy_v)

    if accuracy_v > best_accuracy:
        best_accuracy = accuracy_v
        best_epoch = E
        # Save the model
        torch.save(model, 'best_model.pth')
        counter = 0
    else:
        counter += 1

    if counter >= patience:
        print('Early stopping. No improvement in validation accuracy for {} epochs.'.format(patience))
        break

    if (E+1) % 5 == 0:
        plt.plot(range(len(acc_t)), acc_t, 'b', range(len(acc_v)), acc_v, 'r')
        blue_patch = mpatches.Patch(color='blue', label='Train Accuracy')
        red_patch = mpatches.Patch(color='red', label='Validation Accuracy')
        plt.legend(handles=[red_patch, blue_patch])
        plt.show()

print('Best Validation Accuracy: {} achieved at epoch {}'.format(best_accuracy, best_epoch))

In [ ]:
torch.save(model,'/content/drive/MyDrive/Colab Notebooks/DL/Homework/multi_best_model.pth')

In [ ]:
torch.save(model.state_dict(),'/content/drive/MyDrive/Colab Notebooks/DL/Homework/multi_best_model_state_dict.pth')

# Test

In [ ]:
test_path = '/content/drive/MyDrive/Colab Notebooks/DL/Homework/Hair Diseases - Final/test'

IMG_SIZE = 256

test_data = []

for disease_folder in os.listdir(test_path):
    disease_path = os.path.join(test_path, disease_folder)

    # 디렉토리인지 확인하고 jpg 파일 수집
    if os.path.isdir(disease_path):
        for file_name in os.listdir(disease_path):
            if file_name.endswith('.jpg'):
                # 이미지 읽기
                img_path = os.path.join(disease_path, file_name)
                img = Image.open(img_path)
                # 이미지 리사이즈
                img = img.resize((IMG_SIZE, IMG_SIZE))
                # PIL 이미지를 numpy 배열로 변환
                img_np = np.array(img)
                # 노이즈 제거
                median_filtered_image = cv2.medianBlur(img_np, 3)
                one_img = np.asarray(np.float32(median_filtered_image))
                norm_img = one_img / 255.0

                # 데이터에 추가
                test_data.append(norm_img)

# 테스트 데이터를 numpy 배열로 변환
test_data = np.array(test_data)

# 테스트 데이터 확인
print("Test Data shape:", test_data.shape)

Test Data shape: (1200, 256, 256, 3)


In [ ]:
test_data = test_data.transpose((0, 3, 1, 2))

In [ ]:
# 데이터 확인
print("Test Data shape:", test_data.shape)

Test Data shape: (1200, 3, 256, 256)


In [ ]:
model_path = '/content/drive/MyDrive/Colab Notebooks/DL/Homework/multi_best_model.pth'

In [ ]:
# 모델 정의
class MultiBinaryClassificationModel(nn.Module):
    def __init__(self, num_classes=10) -> None:
        super(MultiBinaryClassificationModel, self).__init__()
        self.resnet = resnet50(pretrained=True)
        # 다중 이진 분류를 위한 출력 레이어 - 뉴런 수를 클래스의 개수에 맞게 조정하고 Sigmoid 적용
        self.classifier = nn.Linear(1000, num_classes)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.resnet(x)
        x = self.classifier(x)
        x = self.sigmoid(x)  # Sigmoid 적용

        return x

# 현재 사용 가능한 디바이스 확인
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 테스트 데이터를 모델 입력 형식에 맞게 변환
test_tensor = torch.from_numpy(test_data).float().to(device)

# 모델 불러오기
loaded_model = torch.load(model_path, map_location=device)
loaded_model.eval()

# 모델과 입력 데이터를 동일한 디바이스로 이동
loaded_model = loaded_model.to(device)

<ipython-input-35-95b5262c17bc>:24: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loaded_model = torch.load(model_path, map_location=device)


In [ ]:
predictions_list = []

# Set the threshold value
threshold = 0.5

for disease_folder in os.listdir(test_path):
    disease_path = os.path.join(test_path, disease_folder)

    if os.path.isdir(disease_path):
        for file_name in os.listdir(disease_path):
            if file_name.endswith('.jpg'):
                img_path = os.path.join(disease_path, file_name)
                img = Image.open(img_path)
                img = img.resize((IMG_SIZE, IMG_SIZE))
                img_np = np.array(img)
                median_filtered_image = cv2.medianBlur(img_np, 3)
                one_img = np.asarray(np.float32(median_filtered_image))
                norm_img = one_img / 255.0
                test_data = np.array([norm_img])
                test_data = test_data.transpose((0, 3, 1, 2))
                test_tensor = torch.from_numpy(test_data).float().to(device)

                with torch.no_grad():
                    outputs = loaded_model(test_tensor)

                predictions = (outputs.cpu().numpy() > threshold).astype(int)  # Apply thresholding
                predictions_list.append(predictions)

# Combine predictions into a numpy array
predictions_array = np.concatenate(predictions_list, axis=0)

# Create a DataFrame
df = pd.DataFrame(predictions_array, columns=['Alopecia Areata', 'Contact Dermatitis', 'Folliculitis', 'Head Lice',
                                              'Lichen Planus', 'Male Pattern Baldness', 'Psoriasis',
                                              'Seborrheic Dermatitis', 'Telogen Effluvium', 'Tinea Capitis'])

# Add image file paths to the DataFrame
image_paths = [os.path.join(disease_path, file_name) for disease_folder in os.listdir(test_path) for file_name in os.listdir(os.path.join(test_path, disease_folder)) if file_name.endswith('.jpg')]
df.insert(0, 'Image_Path', image_paths)

# Save the DataFrame to a CSV file
csv_file_path = '/content/drive/MyDrive/Colab Notebooks/DL/Homework/predictions.csv'
df.to_csv(csv_file_path, index=False)

# Display the path to the saved CSV file
print(f"Predictions are saved at the following path: {csv_file_path}")

Predictions are saved at the following path: /content/drive/MyDrive/Colab Notebooks/DL/Homework/predictions.csv


In [ ]:
# 생성된 CSV 파일의 경로
csv_pred_path = '/content/drive/MyDrive/Colab Notebooks/DL/Homework/predictions.csv'

# CSV 파일 읽기
df_pred = pd.read_csv(csv_pred_path)
df_pred

,Image_Path,Alopecia Areata,Contact Dermatitis,Folliculitis,Head Lice,Lichen Planus,Male Pattern Baldness,Psoriasis,Seborrheic Dermatitis,Telogen Effluvium,Tinea Capitis
0,/content/drive/MyDrive/Colab Notebooks/DL/Home...,1,0,0,0,0,0,0,0,0,0
1,/content/drive/MyDrive/Colab Notebooks/DL/Home...,1,0,0,0,0,0,0,0,0,0
2,/content/drive/MyDrive/Colab Notebooks/DL/Home...,1,0,0,0,0,0,0,0,0,0
3,/content/drive/MyDrive/Colab Notebooks/DL/Home...,1,0,0,0,0,0,0,0,0,0
4,/content/drive/MyDrive/Colab Notebooks/DL/Home...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
1195,/content/drive/MyDrive/Colab Notebooks/DL/Home...,0,0,0,0,0,0,0,0,0,0
1196,/content/drive/MyDrive/Colab Notebooks/DL/Home...,0,0,0,0,0,0,0,0,0,1
1197,/content/drive/MyDrive/Colab Notebooks/DL/Home...,0,0,0,0,0,0,0,0,0,0
1198,/content/drive/MyDrive/Colab Notebooks/DL/Home...,0,0,0,0,0,0,0,0,0,1


In [ ]:
# 생성된 CSV 파일의 경로
csv_test_path = '/content/drive/MyDrive/Colab Notebooks/DL/Homework/Hair Diseases - Final/test_images.csv'

# CSV 파일 읽기
df_test = pd.read_csv(csv_test_path)
df_test

,File Name,Alopecia Areata,Contact Dermatitis,Folliculitis,Head Lice,Lichen Planus,Male Pattern Baldness,Psoriasis,Seborrheic Dermatitis,Telogen Effluvium,Tinea Capitis
0,alopecia_areata_0026,1,0,0,0,0,0,0,0,0,0
1,alopecia_areata_0036,1,0,0,0,0,0,0,0,0,0
2,alopecia_areata_0037,1,0,0,0,0,0,0,0,0,0
3,alopecia_areata_0051,1,0,0,0,0,0,0,0,0,0
4,alopecia_areata_0062,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
1195,tinea_capitis_1147,0,0,0,0,0,0,0,0,0,1
1196,tinea_capitis_1154,0,0,0,0,0,0,0,0,0,1
1197,tinea_capitis_1162,0,0,0,0,0,0,0,0,0,1
1198,tinea_capitis_1170,0,0,0,0,0,0,0,0,0,1


In [ ]:
# Load the ground truth (csv_test_path) and predicted labels (csv_pred_path)
df_test = pd.read_csv(csv_test_path)
df_pred = pd.read_csv(csv_pred_path)

# Extract labels from DataFrames
labels_test = df_test.iloc[:, 1:].values  # Assuming labels start from the second column
labels_pred = df_pred.iloc[:, 1:].values  # Assuming labels start from the second column

# Calculate accuracy
correct_predictions = np.sum(labels_test == labels_pred)
total_samples = labels_test.size
accuracy = correct_predictions / total_samples

print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 98.83%
